In [2]:
import numpy as np
import cv2
#import serial
import time


fire_cascade = cv2.CascadeClassifier('fire_detection.xml')
#fire_detection.xml file & this code should be in the same folder while running the code

#ser1 = serial.Serial('COM14',9600)#change COM port number on which your arduino is connected

cap = cv2.VideoCapture(0)
while 1:
    #ser1.write('0')
    ret, img = cap.read()
    #cv2.imshow('imgorignal',img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    fire = fire_cascade.detectMultiScale(img, 1.2, 5)
    for (x,y,w,h) in fire:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        print('Fire is detected..!')
        #ser1.write('p')
        time.sleep(0.2)
        
    cv2.imshow('img',img)
    #ser1.write('s')
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!
Fire is detected..!


In [1]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score, classification_report

C:\Users\Ocean\Anaconda2\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
size=128
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(size,size,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

In [3]:
model.compile(optimizer=optimizers.RMSprop(lr=0.0003), loss='binary_crossentropy', metrics=['acc'])

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1.255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory('data/train',
target_size=(size,size),batch_size=64, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory('data/valid', target_size=(size,size), batch_size=128, class_mode='binary')
test_generator = test_datagen.flow_from_directory('data/test', target_size=(size,size), batch_size=64, class_mode='binary')

Found 2800 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [6]:
model.fit_generator(train_generator, epochs=5, steps_per_epoch=63,
                    validation_data=validation_generator, validation_steps=7, workers=4)

Epoch 1/5
63/63 [==============================] - 92s 1s/step - loss: 7.9079 - acc: 0.5040 - val_loss: 7.8822 - val_acc: 0.5056
Epoch 2/5
63/63 [==============================] - 77s 1s/step - loss: 8.0451 - acc: 0.4954 - val_loss: 8.1275 - val_acc: 0.4902
Epoch 3/5
63/63 [==============================] - 76s 1s/step - loss: 7.9568 - acc: 0.5009 - val_loss: 7.9907 - val_acc: 0.4988
Epoch 4/5
63/63 [==============================] - 77s 1s/step - loss: 8.0082 - acc: 0.4977 - val_loss: 7.8644 - val_acc: 0.5067
Epoch 5/5
63/63 [==============================] - 77s 1s/step - loss: 7.8764 - acc: 0.5059 - val_loss: 7.8930 - val_acc: 0.5049


In [7]:
model.save('model4.h5')

In [19]:
model_loaded = models.load_model('model.h5')